# Setup

In [1]:
%%capture
!pip install datasets evaluate transformers
!pip install accelerate

In [2]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer,Seq2SeqTrainingArguments
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
import pandas as pd
from datasets import load_metric
import torch
import numpy as np
import re

import warnings
warnings.filterwarnings("ignore")

2024-05-14 01:22:21.298404: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 01:22:21.298531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 01:22:21.418787: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")
model.to('cuda')

tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

# Tokenize data

In [4]:
def tokenize_texts(texts):
    model_inputs = tokenizer(
        texts["inputs"], max_length=1024, truncation=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            texts["labels"], max_length=1024, truncation=True
        )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

def load_and_tokenize_data(data):
    input_lines = data.iloc[:, 0].to_numpy()
    label_lines = data.iloc[:, 1].to_numpy()
    dict_obj = {'inputs': input_lines, 'labels': label_lines}
    dataset = Dataset.from_dict(dict_obj)
    tokenized_dataset = dataset.map(tokenize_texts, batched=True, remove_columns=['inputs'], num_proc=10)

    return tokenized_dataset

In [5]:
train_set = pd.read_csv("/kaggle/input/temp-data/train_set.csv").astype("str")
val_set = pd.read_csv("/kaggle/input/temp-data/val_set.csv").astype("str")
test_set = pd.read_csv("/kaggle/input/temp-data/test_set.csv").astype("str")

In [ ]:
tokenized_train_set = load_and_tokenize_data(train_set)
tokenized_val_set = load_and_tokenize_data(val_set)
tokenized_test_set = load_and_tokenize_data(test_set)

# 3th

## Training

In [16]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

training_args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working/3th",
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    learning_rate=1e-5,
    warmup_ratio=0.05,
    weight_decay=0.01,
    per_device_train_batch_size=9,
    per_device_eval_batch_size=4,
    logging_dir='/kaggle/working/3th',
    group_by_length=True,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=1,
    evaluation_strategy="steps",
    eval_steps=500,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_set,
    data_collator=data_collator,
    eval_dataset=tokenized_val_set
)

In [12]:
import wandb
wandb.login(key="acc398b6c572515cdbf0044bc49909a609a7ba19")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin



wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.



wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.



wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [13]:
trainer.train()

wandb: Currently logged in as: huynhduykhoi619. Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.



  warnings.warn('Was asked to gather along dimension 0, but all '


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.



  warnings.warn('Was asked to gather along dimension 0, but all '



/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.



  warnings.warn('Was asked to gather along dimension 0, but all '



/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.



  warnings.warn('Was asked to gather along dimension 0, but all '



/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scala

TrainOutput(global_step=25539, training_loss=0.02352539929802544, metrics={'train_runtime': 18310.823, 'train_samples_per_second': 25.103, 'train_steps_per_second': 1.395, 'total_flos': 2.431569332754432e+16, 'train_loss': 0.02352539929802544, 'epoch': 3.0})

In [14]:
!zip -r 3th.zip /kaggle/working/3th

  adding: kaggle/working/3rd/ (stored 0%)



  adding: kaggle/working/3rd/checkpoint-25500/ (stored 0%)



  adding: kaggle/working/3rd/checkpoint-25500/rng_state.pth (deflated 25%)



  adding: kaggle/working/3rd/checkpoint-25500/generation_config.json (deflated 29%)



  adding: kaggle/working/3rd/checkpoint-25500/scheduler.pt (deflated 55%)



  adding: kaggle/working/3rd/checkpoint-25500/config.json (deflated 48%)



  adding: kaggle/working/3rd/checkpoint-25500/optimizer.pt (deflated 9%)



  adding: kaggle/working/3rd/checkpoint-25500/trainer_state.json (deflated 83%)



  adding: kaggle/working/3rd/checkpoint-25500/training_args.bin (deflated 51%)



  adding: kaggle/working/3rd/checkpoint-25500/model.safetensors (deflated 7%)



  adding: kaggle/working/3rd/events.out.tfevents.1708473449.744285104d44.34.0 (deflated 68%)


## Evaluate

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/input/vit5-model-3thh")
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
def remove_num_and_punc(text):
    result = re.sub(r'[^\w\s]', '', text)
    result = re.sub(r'\b\w*\d+\w*\b', '', result)
    return result

def accuracy(a, b):
    a = remove_num_and_punc(a).split()
    b = remove_num_and_punc(b).split()
    temp = b.copy()
    count = 0

    for i in a:
        if i in temp:
            temp.remove(i)
            count += 1
    if len(a) > len(b):
        return count/len(b)*0.8
    elif len(a) < len(b):
        return count/len(b)*0.6
    else:
        return count/len(b)

max_target_length = 1024
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")
dataloader = torch.utils.data.DataLoader(tokenized_test_set, collate_fn=data_collator, batch_size=64)

predictions = []
references = []
acc = []

for i, batch in enumerate(tqdm(dataloader)):
    outputs = model.generate(
        input_ids=batch['input_ids'].to('cuda'),
        max_length=max_target_length,
        attention_mask=batch['attention_mask'].to('cuda'),
    )
    
    with tokenizer.as_target_tokenizer():
        outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

        labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
        actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
        
    predictions.extend(outputs)
    references.extend(actuals)

for prediction, reference in zip(predictions, references):
    acc.append(accuracy(prediction, reference))

In [21]:
avg_accuracy = np.mean(np.array(acc).astype(np.float32), axis=0)
print(f"Accuracy: {avg_accuracy * 100}")

Accuracy: 98.07454634249211


## Generate text

In [24]:
from unidecode import unidecode

sentence = unidecode(input())
print(sentence)
encoding = tokenizer(sentence, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=1024,
)
for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

 xin chào thế giới


xin chao the gioi
Xin chào thế giới
